In [38]:
from langchain_neo4j import  Neo4jGraph
from dotenv import load_dotenv

load_dotenv()



True

In [40]:
graph = Neo4jGraph()


import json

with open("../relations.json", "r", encoding="utf-8") as f:
    data = json.load(f)


In [4]:
query = """
MERGE (f:Function {name: $fun_name})
SET f.code = $fun_code
MERGE (fi:File {name: $file_name})
MERGE (f)-[:DEFINED_IN]->(fi)
"""


for x in data['functions']:
    graph.query(query=query , params={
        "fun_name" : x,
        "file_name" : data['functions'][x],
        "fun_code" : data['function_code'][x],
    })


In [5]:
query_funCall = """
MERGE (f:Function {name: $Caller})
MERGE (fi:Function {name: $Calle})
MERGE (f)-[:Call]->(fi)
"""
node_count = 0
for x in data['function_calls']:
   for y in data['function_calls'][x]:
        graph.query(query=query_funCall , params={
            "Caller" : x,
            "Calle" : y
        })
        print("Realtion Created successFully" , node_count)
        node_count+=1

Realtion Created successFully 0
Realtion Created successFully 1
Realtion Created successFully 2
Realtion Created successFully 3
Realtion Created successFully 4
Realtion Created successFully 5
Realtion Created successFully 6
Realtion Created successFully 7
Realtion Created successFully 8
Realtion Created successFully 9
Realtion Created successFully 10
Realtion Created successFully 11
Realtion Created successFully 12
Realtion Created successFully 13
Realtion Created successFully 14
Realtion Created successFully 15
Realtion Created successFully 16
Realtion Created successFully 17
Realtion Created successFully 18
Realtion Created successFully 19
Realtion Created successFully 20
Realtion Created successFully 21
Realtion Created successFully 22
Realtion Created successFully 23
Realtion Created successFully 24
Realtion Created successFully 25
Realtion Created successFully 26
Realtion Created successFully 27
Realtion Created successFully 28
Realtion Created successFully 29
Realtion Created suc

In [6]:
query_funCall = """
MERGE (f:Function {name: $Caller})
MERGE (fi:Function {name: $Calle})
MERGE (fi)-[:USED_IN]->(f)
"""
node_count = 0
for x in data['function_calls']:
   for y in data['function_calls'][x]:
        graph.query(query=query_funCall , params={
            "Caller" : x,
            "Calle" : y
        })
        print("Realtion Created successFully" , node_count)
        node_count+=1

Realtion Created successFully 0
Realtion Created successFully 1
Realtion Created successFully 2
Realtion Created successFully 3
Realtion Created successFully 4
Realtion Created successFully 5
Realtion Created successFully 6
Realtion Created successFully 7
Realtion Created successFully 8
Realtion Created successFully 9
Realtion Created successFully 10
Realtion Created successFully 11
Realtion Created successFully 12
Realtion Created successFully 13
Realtion Created successFully 14
Realtion Created successFully 15
Realtion Created successFully 16
Realtion Created successFully 17
Realtion Created successFully 18
Realtion Created successFully 19
Realtion Created successFully 20
Realtion Created successFully 21
Realtion Created successFully 22
Realtion Created successFully 23
Realtion Created successFully 24
Realtion Created successFully 25
Realtion Created successFully 26
Realtion Created successFully 27
Realtion Created successFully 28
Realtion Created successFully 29
Realtion Created suc

In [ ]:
query_Class = """
MERGE (f:Class {name: $Class})
MERGE (fi:File {name: $file_name})
MERGE (f)-[:DEFINED_IN]->(fi)
"""
node_count = 0
for x in data['classes']:
    graph.query(query=query_Class , params={
        "Class" : x,
        "file_name" : data['classes'][x]['file']
    })
    print("Realtion Created successFully" , node_count)
    node_count+=1

In [8]:
query_Class_In = """
MERGE (f:Class {name: $Class})
MERGE (c:Class {name: $Base})
MERGE (f)-[:INHERITS_FROM]->(c)
"""
node_count = 0
for x in data['classes']:
  for y in data['classes'][x]['bases']:
    graph.query(query=query_Class_In , params={
        "Class" : x,
        "Base" : y
    })
    print(x, y)
    node_count+=1

User Base
Video Base
VideoBase BaseModel
VideoCreate VideoBase
VideoResponse VideoBase
UserBase BaseModel
UserLogin BaseModel
UserCreate UserBase
UserResponse UserBase
Token BaseModel
TokenData BaseModel
ScenePlan BaseModel
ManimCodeResponse BaseModel
ManimExecutionResponse BaseModel
ManimErrorCorrectionResponse BaseModel


In [ ]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
import re
from langchain.prompts import PromptTemplate

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.0,
)

KG_SCHEMA = """
You are working with a Neo4j knowledge graph with the following structure:

Node Types:
- Function {name, code}
- File {name}
- Class {name}

Relationships:
- (Function)-[:DEFINED_IN]->(File)
- (Function)-[:Call]->(Function)
- (Function)<-[:USED_IN]-(Function)
- (Class)-[:DEFINED_IN]->(File)
- (Class)-[:INHERITS_FROM]->(Class)

Each Function node has:
- `name`: the function name
- `code`: the function's full source code

Each function is connected to the file it is defined in via:
- (Function)-[:DEFINED_IN]->(File)

Functions can call each other via:
- (caller:Function)-[:Call]->(callee:Function)

---

You will be asked natural language questions. Your task is to **only generate valid Cypher queries** that return:

1. The function being queried (callee):  
   - its name  
   - the file it's defined in  
   - its source code

2. All caller functions (those that call the above function):  
   - each caller's name  
   - the file it is defined in  
   - its source code

---

Use this Cypher template:

```cypher
MATCH (callee:Function {name: "func_name"})-[:DEFINED_IN]->(calleeFile:File)
OPTIONAL MATCH (caller:Function)-[:Call]->(callee)
OPTIONAL MATCH (caller)-[:DEFINED_IN]->(callerFile:File)
RETURN
  callee.name AS target_name,
  calleeFile.name AS target_file,
  callee.code AS target_code,
  collect({name: caller.name, file: callerFile.name, code: caller.code}) AS callers
"""

cypher_prompt = PromptTemplate(
    input_variables=["schema", "question"],
    template="""
{schema}

Question: {question}

Cypher:
"""
)

def generate_cypher_query(user_question):

    full_prompt = cypher_prompt.format(schema=KG_SCHEMA, question=user_question)
    result = llm.invoke([HumanMessage(content=full_prompt)])
    
    match = re.search(r"```cypher(.*?)```", result.content, re.DOTALL)
    cypher = match.group(1).strip() if match else result.content.strip()
    return cypher

def run_query(cypher_query):
    try:
        result = graph.query(cypher_query)
        return result
    except Exception as e:
        return {"error": str(e)}


user_input = "Give all caller of create_access_token function"

cypher = generate_cypher_query(user_input)

result = run_query(cypher) 


'def create_access_token(data: dict, expires_delta: timedelta | None = None) -> str:\n    to_encode = data.copy()\n    expire = datetime.utcnow() + (expires_delta or timedelta(minutes=15))\n    to_encode.update({"exp": expire})\n    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)\n'

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    input_variables=['question' , 'func_code' , 'fun_loc', 'caller_code'],

    template=""" 
    You are strong Python developer who will assist user to do changes based on there question you are provided current function code and other functions which call 
    this function modify the function based on user question  also modify dependent functions if needed !
    User : {question},
    function_code : {func_code}
    function_defined in : {fun_loc}
    Caller's code : {caller_code}
"""
)

llm = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash",
    temperature = 0.5
)

chain = prompt | llm | StrOutputParser()

def format_callers(callers):
    return "\n\n".join(
        f"# {c['name']} (in {c['file']})\n```python\n{c['code']}\n```"
        for c in callers
    )

formatted_callers = format_callers(result[0]["callers"])

response = chain.invoke({
    "question": "Can you add data validation in this create token function?",
    "func_code": result[0]["target_code"],
    "fun_loc": result[0]["target_file"],
    "caller_code": formatted_callers
})




In [37]:

print(response)

```python
from typing import Dict, Optional, Union
from datetime import datetime, timedelta
from jose import jwt
from pydantic import BaseModel, ValidationError

# Define a Pydantic model for the token data
class TokenData(BaseModel):
    sub: str  # Subject (usually the username or user ID)
    exp: Optional[datetime] = None  # Expiration time (optional, will be added later)
    # You can add other fields here as needed, and they will be validated

def create_access_token(data: Dict[str, str], expires_delta: Optional[timedelta] = None) -> str:
    """
    Creates an access token with data validation using Pydantic.

    Args:
        data: A dictionary containing the data to be encoded in the token.
        expires_delta: An optional timedelta object representing the expiration time.

    Returns:
        A JWT access token.

    Raises:
        ValueError: If the data does not conform to the TokenData model.
    """
    try:
        token_data = TokenData(**data)  # Validate the inpu